In [1]:
#pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
directory = "TImages/"

In [4]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [5]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [6]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [7]:
create_training_data()

In [8]:
lenofimage = len(training_data)
print(lenofimage)

5850


In [9]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, )

In [12]:
## SVM MODEL

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [14]:
# Feature Scaling
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [20]:
# Hyperparameter Tuning using GridSearchCV
# param_grid = {'C': [0.1, 1, 10],
#               'kernel': ['linear', 'rbf', 'poly'],
#               'gamma': ['scale', 'auto', 0.1, 1, 10]
#               }
param_grid = {'C': [1,10,30],
              'kernel': ['linear'],
              'gamma': ['auto']
              }

In [21]:
# Train the SVM model
svm = SVC()
grid_search = GridSearchCV(svm, param_grid, cv=5, verbose=2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END .....................C=1, gamma=auto, kernel=linear; total time= 8.1min
[CV] END .....................C=1, gamma=auto, kernel=linear; total time= 8.0min
[CV] END .....................C=1, gamma=auto, kernel=linear; total time= 8.8min
[CV] END .....................C=1, gamma=auto, kernel=linear; total time= 8.0min
[CV] END .....................C=1, gamma=auto, kernel=linear; total time= 8.4min
[CV] END ....................C=10, gamma=auto, kernel=linear; total time= 7.8min
[CV] END ....................C=10, gamma=auto, kernel=linear; total time= 7.3min
[CV] END ....................C=10, gamma=auto, kernel=linear; total time= 7.9min
[CV] END ....................C=10, gamma=auto, kernel=linear; total time= 7.1min
[CV] END ....................C=10, gamma=auto, kernel=linear; total time= 7.5min
[CV] END ....................C=30, gamma=auto, kernel=linear; total time= 7.2min
[CV] END ....................C=30, gamma=auto, ke

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 10, 30], 'gamma': ['auto'],
                         'kernel': ['linear']},
             verbose=2)

In [22]:
# Get the best estimator and evaluate it
best_svm = grid_search.best_estimator_
y_test_pred = best_svm.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best SVM Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Best SVM Parameters: {'C': 1, 'gamma': 'auto', 'kernel': 'linear'}
Accuracy: 90.26%


In [ ]:
# work progressing...........


In [24]:
grid_search.cv_results_

{'mean_fit_time': array([380.81848927, 369.53007994, 348.83321815]),
 'std_fit_time': array([13.72440795, 15.54798096, 30.24162519]),
 'mean_score_time': array([115.95088134,  81.29342189,  66.24521995]),
 'std_score_time': array([ 8.25907079, 17.35701046,  4.80347485]),
 'param_C': masked_array(data=[1, 10, 30],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_gamma': masked_array(data=['auto', 'auto', 'auto'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'linear', 'linear'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'gamma': 'auto', 'kernel': 'linear'},
  {'C': 10, 'gamma': 'auto', 'kernel': 'linear'},
  {'C': 30, 'gamma': 'auto', 'kernel': 'linear'}],
 'split0_test_score': array([0.94551282, 0.94551282, 0.94551282]),
 'split1_test_score': array([0.94444444, 0.944444

In [26]:
df = pd.DataFrame(grid_search.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,380.818489,13.724408,115.950881,8.259071,1,auto,linear,"{'C': 1, 'gamma': 'auto', 'kernel': 'linear'}",0.945513,0.944444,0.946581,0.951923,0.946581,0.947009,0.002582,1
1,369.530080,15.547981,81.293422,17.357010,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.945513,0.944444,0.946581,0.951923,0.946581,0.947009,0.002582,1
2,348.833218,30.241625,66.245220,4.803475,30,auto,linear,"{'C': 30, 'gamma': 'auto', 'kernel': 'linear'}",0.945513,0.944444,0.946581,0.951923,0.946581,0.947009,0.002582,1


In [28]:
df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.947009
1,10,linear,0.947009
2,30,linear,0.947009


In [30]:
grid_search.best_score_

0.947008547008547

In [31]:
grid_search.best_params_

{'C': 1, 'gamma': 'auto', 'kernel': 'linear'}